## Data Processing Peak Season

In [3]:
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats
from nba_api.stats.static import players
from college_info_puller import CollegeInfo
import pandas as pd
import numpy as np
import pickle
import time
import requests

In [172]:
nba_03 = pd.read_csv('Data/season_dat/NBA_2003-04.csv')
nba_04 = pd.read_csv('Data/season_dat/NBA_2004-05.csv')
nba_05 = pd.read_csv('Data/season_dat/NBA_2005-06.csv')
nba_06 = pd.read_csv('Data/season_dat/NBA_2006-07.csv')
nba_07 = pd.read_csv('Data/season_dat/NBA_2007-08.csv')
nba_08 = pd.read_csv('Data/season_dat/NBA_2008-09.csv')
nba_09 = pd.read_csv('Data/season_dat/NBA_2009-10.csv')
nba_10 = pd.read_csv('Data/season_dat/NBA_2010-11.csv')
nba_11 = pd.read_csv('Data/season_dat/NBA_2011-12.csv')
nba_12 = pd.read_csv('Data/season_dat/NBA_2012-13.csv')
nba_13 = pd.read_csv('Data/season_dat/NBA_2013-14.csv')
nba_14 = pd.read_csv('Data/season_dat/NBA_2014-15.csv')
nba_15 = pd.read_csv('Data/season_dat/NBA_2015-16.csv')
nba_16 = pd.read_csv('Data/season_dat/NBA_2016-17.csv')
nba_17 = pd.read_csv('Data/season_dat/NBA_2017-18.csv')
nba_18 = pd.read_csv('Data/season_dat/NBA_2018-19.csv')
nba_19 = pd.read_csv('Data/season_dat/NBA_2019-20.csv')
nba_20 = pd.read_csv('Data/season_dat/NBA_2020-21.csv')
nba_21 = pd.read_csv('Data/season_dat/NBA_2021-22.csv')
nba_22 = pd.read_csv('Data/season_dat/NBA_2022-23.csv')
nba_23 = pd.read_csv('Data/season_dat/NBA_2023-24.csv')
nba_24 = pd.read_csv('Data/season_dat/NBA_2024-25.csv')

nba = pd.concat([nba_03,nba_04,nba_05,nba_06,nba_07,nba_08,nba_09,nba_10,
                 nba_11,nba_12,nba_13,nba_14,nba_15,nba_16,nba_17,nba_18,nba_19,nba_20,
                 nba_21,nba_22,nba_23,nba_24], ignore_index=True, sort=False)

Let's get career data for those who have played in the NBA recently. Say over the last 10 years who were the players who got some playing time? The cut off will be set at 250 min a season or 3 min per game to allow for some fringe players or those who got injured to be included in our data set. 

Then we'll pull the players peak years as a measure of their true ability

In [173]:
nba_recent = pd.concat([nba_24,nba_23,nba_22,nba_21,nba_20,nba_19,nba_18,nba_17,nba_16,nba_15], ignore_index=True, sort=False)
rec_fil = nba_recent[nba_recent['MP']>=250]
cur_players = rec_fil.groupby('Player')['MP'].sum().sort_values().index
all_players = nba.groupby('Player')['MP'].sum().index

On full data need to similarily filter out rows less than our MP cutoff

In [177]:
nba = nba[nba['MP']>=250]

In [178]:
nba_bpms= nba.groupby('Player')['BPM'].unique()
nba_ages = nba.groupby('Player')['Age'].unique()

#best season by bpm
bpm_maxes = []

#will take average of up to a players 3 best bpm seasons, avg of 2 or 1 if they haven't been in league that long
avg_best_bpms = []

#starting age when debuting
age_enter = []

#age when peaked
age_peaked = []

for p in cur_players:
    if p in all_players:
        bpms = nba_bpms.loc[p]
        peaked = max(bpms)
        peaked_loc = bpms.argmax()
        
        bpm_maxes.append(peaked)
        
        #up to
        top3_bpms = np.sort(bpms)[::-1][:3]
        
        avg_best_bpms.append(np.mean(top3_bpms))
        
        ages = nba_ages.loc[p]
        age_enter.append(min(ages))

        age_peaked.append(ages[peaked_loc])
        
        
peaked_df = pd.DataFrame({'Player': cur_players,
              'Age Enter': age_enter,
              'Max BPM': bpm_maxes,
              'Age Peaked': age_peaked,
              'Top 3 BPM Avg': avg_best_bpms})
        

In [140]:
info_dict = CollegeInfo().info_dict

player_w_info = list(info_dict.keys())

missing_info = []

for player in peaked_df['Player']:
    if player not in player_w_info:
        missing_info.append(player)

In [146]:
collegefetcher = CollegeInfo()

updated_info = collegefetcher.fill_in_info(
    cur_players=missing_info,
    player_w_info=player_w_info
)

Could not find: R.J. Hunter
Could not find: J.J. Hickson
Could not find: K.J. McDaniels
Could not find: P.J. Hairston
Could not find: J.R. Smith
Saving pulled info...


In [153]:
retry_list = ['RJ Hunter', 'JJ Hickson', 'KJ McDaniels', 'PJ Hairston', 'JR Smith']

updated_info = collegefetcher.fill_in_info(
    cur_players=retry_list,
    player_w_info=player_w_info
)

Saving pulled info...


In [ ]:
info_dict = collegefetcher.info_dict

del info_dict['J.R. Smith']
del info_dict['R.J. Hunter']
del info_dict['J.J. Hickson']
del info_dict['K.J. McDaniels']
del info_dict['P.J. Hairston']

with open(collegefetcher.info_dict_path, 'wb') as f:
    pickle.dump(info_dict, f)
    


In [192]:
updated_info = collegefetcher.fill_in_info(
    cur_players=['R.J. Hampton'],
    player_w_info=player_w_info
)

Saving pulled info...


In [ ]:
players.find_players_by_full_name('R.J. Hampton')

player_info = collegefetcher.get_player_info(1630181)

info = player_info.common_player_info.get_data_frame()

college = info.loc[0, "SCHOOL"]
draft_pick = info.loc[0, "DRAFT_NUMBER"]

info_dict['R.J. Hampton'] = {
    'College': college,
    'Draft Pick': draft_pick
}

In [201]:
with open(collegefetcher.info_dict_path, 'wb') as f:
    pickle.dump(info_dict, f)

## Adding Draft Features

In [184]:
rename = {'R.J. Hunter': 'RJ Hunter',
          'J.J. Hickson': 'JJ Hickson',
          'K.J. McDaniels': 'KJ McDaniels',
          'P.J. Hairston': 'PJ Hairston',
          'J.R. Smith': 'JR Smith'}

peaked_df['Player'] = peaked_df['Player'].replace(rename)

In [180]:
collegefetcher = CollegeInfo()

info_dict = collegefetcher.info_dict

In [185]:
peaked_df['College'] = peaked_df['Player'].map(lambda x: info_dict[x]['College'])
peaked_df['Draft Pick'] = peaked_df['Player'].map(lambda x: info_dict[x]['Draft Pick'])

In [186]:
peaked_df.loc[0,'Draft Pick']
peaked_df.loc[peaked_df['Draft Pick'].isnull(),'Draft Pick'] = 0
peaked_df['Draft Pick'] = peaked_df['Draft Pick'].replace({'Undrafted':0})
peaked_df['Draft Pick'] = peaked_df['Draft Pick'].astype(int)

In [187]:
rounds = [] 
round_bins = []

for i in range(len(peaked_df)):
    if peaked_df.loc[i,'Draft Pick'] > 0 and peaked_df.loc[i,'Draft Pick'] <= 30:
        rounds.append('1st')
        if peaked_df.loc[i,'Draft Pick'] <= 14:
            round_bins.append('1-14')
        else:
            round_bins.append('15-30')
    elif peaked_df.loc[i,'Draft Pick'] > 30 and peaked_df.loc[i,'Draft Pick'] <= 60:
        rounds.append('2nd')
        if peaked_df.loc[i,'Draft Pick'] <= 44:
            round_bins.append('31-44')
        else:
            round_bins.append('45-60')
    else:
        rounds.append('Undrafted')
        round_bins.append('Undrafted')

peaked_df['Round'] = rounds
peaked_df['Round Bin'] = round_bins
peaked_df.head()

,Player,Age Enter,Max BPM,Age Peaked,Top 3 BPM Avg,College,Draft Pick,Round,Round Bin
0,Pat Spencer,28,-1.6,28,-1.600000,Northwestern,0,Undrafted,Undrafted
1,Jeremy Evans,23,4.2,27,1.966667,Western Kentucky,55,2nd,45-60
2,Cam Spencer,24,0.5,24,0.500000,Connecticut,53,2nd,45-60
3,Marcus Bagley,23,-4.3,23,-4.300000,Arizona State,0,Undrafted,Undrafted
4,Julian Washburn,27,-4.6,27,-4.600000,Texas-El Paso,0,Undrafted,Undrafted


Manually cleaning up so player data

In [188]:
peaked_df.loc[peaked_df['Player']=='Sekou Doumbouya','College'] = "INSEP"
peaked_df.loc[peaked_df['Player']=="Mãozinha Pereira",'College'] = "Cearense"
peaked_df.loc[peaked_df['Player']=="Alex Ducas",'College'] = "Australian Institute of Sport"
peaked_df.loc[peaked_df['Player']=="Alex Sarr",'College'] = "Perth"
peaked_df.loc[peaked_df['Player']=='Josh Giddey','College'] = "Adelaide"
peaked_df.loc[peaked_df['Player']=='Al Jefferson','College'] = "Prentiss (HS)"
peaked_df.loc[peaked_df['Player']=='Gerald Green','College'] = "Gulf Shores Academy (HS)"
peaked_df.loc[peaked_df['Player']=='JR Smith','College'] = "St. Benedict's Prep (HS)"

Cleaning up College data to be consistent

In [189]:
peaked_df.loc[(peaked_df['College']=="St. John's, N.Y.") | (peaked_df['College']=="St. John's (NY)"), 'College'] = "St. John's"
peaked_df.loc[(peaked_df['College']=="University of Texas at Austin") | (peaked_df['College']=="Texas-Austin"), 'College'] = "Texas"
peaked_df.loc[peaked_df['College']=="Wisc.-Green Bay", 'College'] = "Wisconsin-Green Bay"
peaked_df.loc[peaked_df['College']=="Nebraska-Lincoln", 'College'] = "Nebraska"
peaked_df.loc[(peaked_df['College']=="St. Mary's College") | (peaked_df['College']=="Saint Mary's"), 'College'] = "St. Mary's"
peaked_df.loc[peaked_df['College']=="Capital City Go-Go", 'College'] = "New Zealand Breakers"
peaked_df.loc[peaked_df['College']=="Barcelona, Spain", 'College'] = "Olympiacos"
peaked_df.loc[peaked_df['College']=="Edgewater HS (FL)", 'College'] = "IMG Academy (FL)"
peaked_df.loc[peaked_df['College']=="Lithuania", 'College'] = 'Benetton Treviso'
peaked_df.loc[peaked_df['College']=="Serbia", 'College'] = 'Mega Basket'
peaked_df.loc[peaked_df['College']=="Nevada-Reno", 'College'] = 'Nevada'
peaked_df.loc[peaked_df['College']=='California-Santa Barbara', 'College'] = 'Cal-Santa Barbara'
peaked_df.loc[peaked_df['College']=='NBA G League Ignite', 'College'] = 'G League Ignite'
peaked_df.loc[peaked_df['College']=='Ratiopharm Ulm', 'College'] = 'Ratiopharm'

In [190]:
country_map = {
    'Russia': ['Zenit Saint Petersburg','Khimki','CSKA Moscow'],
    'Greece': ['Olympiacos','Filathlitikos','Panathinaikos'],
    'Argentina': ['Penarol'],
    'Italy': ['Benetton Treviso','Udine','Virtus Roma','Angelico Biella','Fortitudo Bologna','Olimpia Milano','Virtus Bologna'],
    'Spain': ['Malaga','Tau Ceramica',"Baloncesto Fuenlabrada","Baskonia","Bibao","Cajasol Sevilla",
              "Estudiantes","FC Barcelona","Gran Canaria","Murcia","Real Madrid","Ricoh Manresa","San Pablo Burgos","Zaragoza",'Bilbao'],
    'Turkey': ['Darussafaka','Ulkerspor',"Anadolu Efes","Besiktas","Bursaspor","Crvena zvezda","Fenerbahce"],
    'France': ['Hyeres-Toulon','INSEP','Elan Bearnais','Paris Basket Racing',"ADA Blois",
               "ASVEL","Cholet","Elan Chalon","JL Bourg-en-Bresse","Le Mans","Metropolitans 92",
               "Pau Orthez","Poitiers Basket 86","Rouen, France","Strasbourg IG"],
    'Brazil': ['Cearense','Bauru','Vasco de Gama',"Flamengo","Minas","Pinheiros"],
    'Australia': ['Australian Institute of Sport','Perth','Adelaide','New Zealand Breakers',"Brisbane","Illawarra"],
    'Serbia': ['Mega Basket'],
    'Germany': ['DJK Wurzburg','RheinEnergie Cologne','Brose Bamberg',"Bayern Munich","Braunschweig","Ratiopharm","Riesen Ludwigsburg",'Skyliners Frankfurt'],
    'Canada': ['Athlete Institute Prep','Vanier'],
    'Solvenia': ["Cedevita","Union Olimpija"],
    'China': ["Guangdong","Guangzhou Loong Lions"],
    'Lithuania': ["Lietuvos rytas Vilnius","Zalgiris"],
    'Israel': ['Maccabi Tel Aviv'],
    'Senegal': ['NBA Global Academy']
}

league_map = {
    'VTB United League': ['Zenit Saint Petersburg'],
    'JUCO': ['Blinn J.C.'],
    'Other D1': ['Texas-El Paso','Campbell University','Georgia State','Valparaiso','Rider',
                 'Mercy','Towson','Western Carolina','Cleveland State','Utah Valley','Wisconsin-Green Bay',
                 'Eastern Washington','Temple','Va Commonwealth','Nevada','Norfolk State','Harvard',
                 'Northeastern','Vanier','Akron','Alabama-Birmingham','Arkansas-Little Rock','Austin Peay',
                 'Belmont','Boise State','Bowling Green','Bradley','Bucknell','Buffalo','Cal Poly-Obispo',
                 'Cal State-Long Beach','Cal-Davis','Cal-Santa Barbara','California State-Fullerton',
                 'Colorado State','Davidson','Dayton','Delaware','Denver','Detroit Mercy','Drexel','Duquesne',
                 'Eastern Michigan','Florida Gulf Coast','Fresno State','Furman','George Washington','Indiana-Purdue Fort Wayne',
                 'Indiana-Purdue Indianapolis','Iona','John A. Logan','La Salle','Lehigh','Lincoln Memorial','Lipscomb',
                 'Louisana-Lafayette','Louisiana Tech','Loyola-Maryland','Marshall','Montana State','Morehead State',
                 'Murray State','New Mexico','New Mexico State','Northern Iowa','Northwest Missouri St.','Oakland','Ohio',
                 'Old Dominion','Pepperdine','Princeton','Rhode Island','Richmond',"St. Mary's",'San Francisco',
                 'Santa Clara','South Carolina Upstate','St. Bonaventure','St. Louis','Stephen F. Austin',
                 'Tennessee State','Tennessee-Chattanooga','Texas-San Antonio','Toledo','Truman State','Tulane','Tulsa',
                 'UNC-Wilmington','University of Charleston','UNLV','Utah State','Vermont','Virginia Commonwealth','Washington State',
                 'Weber State','Western Kentucky','Wheeling Jesuit','Wichita State','William & Mary','Winthrop','Wisconsin-Milwaukee',
                 'Wyoming','Yale','Detroit','Radford','Wake Forest'],
    'High School': ['Clifton J. Ozen HS (TX)','IMG Academy (FL)','Lower Merion HS (PA)','Prentiss (HS)',
                    'Gulf Shores Academy (HS)',"St. Benedict's Prep (HS)",'Lanier HS (MS)',
                    'Dominguez HS (CA)','Peoria Central HS (IL)','Westchester HS (CA)','Princeton HS (OH)',
                    'Ranney School (NJ)','Skyline HS (TX)','South Gwinnett HS (GA)',"St. Joseph's (PA)",'St. Vincent-St. Mary HS (OH)',
                    'SW Atlanta Christian Academy (GA)','Moravian Prep (NC)','Farragut Academy HS (IL)','Cypress Creek HS (FL)','Oak Hill Academy (VA)'],
    'Euroleague': ['Olympiacos','Benetton Treviso','Tau Ceramica','Ulkerspor','Fortitudo Bologna','Anadolu Efes',
                   'Crvena zvezda','Fenerbahce','Bayern Munich','Baskonia','FC Barcelona','Real Madrid','Panathinaikos','Zalgiris','Olimpia Milano','Virtus Bologna',
                   'CSKA Moscow'],
    'LUB': ['Penarol'],
    'Liga ACB': ['Malaga','Bibao','Cajasol Sevilla','Gran Canaria','Murcia','Ricoh Manresa','San Pablo Burgos','Zaragoza','Bilbao'],
    'BSL': ['Darussafaka'],
    'Serie A': ['Udine'],
    'LNB Pro B': ['Hyeres-Toulon','Elan Bearnais','ADA Blois','Pau Orthez','Poitiers Basket 86','Rouen, France'],
    'International Prep': ['INSEP','Australian Institute of Sport','Athlete Institute Prep','NBA Global Academy'],
    'NBB': ['Cearense','Bauru','Vasco de Gama','Flamengo','Minas','Pinheiros'],
    'NBL': ['Perth','Adelaide','New Zealand Breakers','Brisbane','Illawarra'],
    'Mega Superbasket': ['Mega Basket'],
    'Super League 1': ['Khimki'],
    'Serie B': ['Virtus Roma','Angelico Biella'],
    'LNB Pro A': ['Paris Basket Racing','ASVEL','Cholet','Elan Chalon','JL Bourg-en-Bresse','Le Mans','Strasbourg IG'],
    'Bundesliga': ['DJK Wurzburg','RheinEnergie Cologne','Brose Bamberg','Braunschweig','Ratiopharm','Riesen Ludwigsburg','Skyliners Frankfurt'],
    'Nationale Masculine 1': ['Metropolitans 92'],
    'Super Ligi': ['Besiktas','Bursaspor'],
    'Primera FEB': ['Baloncesto Fuenlabrada','Estudiantes'],
    'ABA': ['Cedevita','Union Olimpija'],
    'Greek B': ['Filathlitikos'],
    'CBA': ['Guangdong','Guangzhou Loong Lions'],
    'LKL': ['Lietuvos rytas Vilnius'],
    'Israeli Premier League': ['Maccabi Tel Aviv'],
    'G League': ['G League Ignite'],
    'Overtime': ['Overtime Elite']
}

countries = []
leagues = []

for i in range(len(peaked_df)):
    player_college = peaked_df.loc[i,'College']
    
    assigned_country = 'USA'
    assigned_league = 'Top D1'
    
    # Lookup country
    for country, teams in country_map.items():
        if player_college in teams:
            assigned_country = country
            break
    
    # Lookup league
    for league, teams in league_map.items():
        if player_college in teams:
            assigned_league = league
            break
    
    countries.append(assigned_country)
    leagues.append(assigned_league)

peaked_df['Country'] = countries
peaked_df['League'] = leagues

In [191]:
league_diff_map = {'Euroleague':1,
                   'Top D1': 2,
                   'Liga ACB': 3,
                   'Serie A': 4,
                   'Super Ligi': 5,
                   'NBL': 6,
                   'Other D1': 7,
                   'G League': 8,
                   'CBA': 9,
                   'LNB Pro A': 10,
                   'LUB': 11,
                   'VTB United League': 12,
                   'Bundesliga': 13,
                   'LKL': 14,
                   'NBB': 15,
                   'Mega Superbasket': 16,
                   'ABA': 17,
                   'Primera FEB': 18,
                   'Serie B': 19,
                   'Greek B': 20,
                   'LNB Pro B': 21,
                   'Israeli Premier League':22,
                   'International Prep': 23,
                   'BSL': 24,
                   'LUB': 25,
                   'Super League 1': 26,
                   'Overtime': 27, 
                   'Nationale Masculine 1': 28,                   
                   'JUCO': 29,
                   'High School': 30
                   }

peaked_df['League Diff'] = peaked_df['League'].map(lambda x: league_diff_map[x])

In [215]:
cur_active = nba_24['Player'].tolist()
players = peaked_df['Player'].tolist()

look_at_values = []

for i, player in enumerate(cur_players):
    if player not in cur_active:
        look_at_values.append(i)

Manually adjusting some values for players with long careers

In [255]:
peaked_df.loc[peaked_df['Player']=='Richard Jefferson','Age Enter'] =  21
peaked_df.loc[peaked_df['Player']=='Jason Terry','Age Enter'] =  22
peaked_df.loc[peaked_df['Player']=='Nenê','Age Enter'] =  20
peaked_df.loc[peaked_df['Player']=='Drew Gooden','Age Enter'] =  23
peaked_df.loc[peaked_df['Player']=='Chris Andersen','Age Enter'] =  23

peaked_df.loc[peaked_df['Player']=='Andre Miller','Age Enter'] =  23
peaked_df.loc[peaked_df['Player']=='Andre Miller','Max BPM'] =  3.9
peaked_df.loc[peaked_df['Player']=='Andre Miller','Age Peaked'] =  25
peaked_df.loc[peaked_df['Player']=='Andre Miller','Top 3 BPM Avg'] =  (3.9+2.8+2.5)/3

peaked_df.loc[peaked_df['Player']=='Kevin Garnett','Age Enter'] =  19
peaked_df.loc[peaked_df['Player']=='Kevin Garnett','Top 3 BPM Avg'] =  (10.2+9.5+8.4)/3

peaked_df.loc[peaked_df['Player']=='Paul Pierce','Age Enter'] =  21
peaked_df.loc[peaked_df['Player']=='Paul Pierce','Max BPM'] =  5.9
peaked_df.loc[peaked_df['Player']=='Paul Pierce','Age Peaked'] =  24
peaked_df.loc[peaked_df['Player']=='Paul Pierce','Top 3 BPM Avg'] =  (5.9+4.9+4.8)/3

peaked_df.loc[peaked_df['Player']=='Tim Duncan','Age Enter'] =  21
peaked_df.loc[peaked_df['Player']=='Tim Duncan','Top 3 BPM Avg'] =  (8.5+7.6+7.6)/3

peaked_df.loc[peaked_df['Player']=='Mike Dunleavy','Age Enter'] =  22

peaked_df.loc[peaked_df['Player']=='Kobe Bryant','Age Enter'] =  18
peaked_df.loc[peaked_df['Player']=='Kobe Bryant','Top 3 BPM Avg'] =  (7.6+7.1+5.9)/3

peaked_df.loc[peaked_df['Player']=='Manu Ginóbili','Age Enter'] =  25
peaked_df.loc[peaked_df['Player']=='Zach Randolph','Age Enter'] =  20
peaked_df.loc[peaked_df['Player']=='Tony Parker','Age Enter'] =  19

peaked_df.loc[peaked_df['Player']=='Vince Carter','Age Enter'] =  22
peaked_df.loc[peaked_df['Player']=='Vince Carter','Max BPM'] =  7.6
peaked_df.loc[peaked_df['Player']=='Vince Carter','Age Peaked'] =  24
peaked_df.loc[peaked_df['Player']=='Vince Carter','Top 3 BPM Avg'] =  (7.6+6+5.5)/3

peaked_df.loc[peaked_df['Player']=='Dirk Nowitzki','Age Enter'] =  20
peaked_df.loc[peaked_df['Player']=='Pau Gasol','Age Enter'] =  21


In [283]:
peaked_df[(peaked_df['Top 3 BPM Avg']<-2)]

,Player,Age Enter,Max BPM,Age Peaked,Top 3 BPM Avg,College,Draft Pick,Round,Round Bin,Country,League,League Diff
3,Marcus Bagley,23,-4.3,23,-4.300000,Arizona State,0,Undrafted,Undrafted,USA,Top D1,2
4,Julian Washburn,27,-4.6,27,-4.600000,Texas-El Paso,0,Undrafted,Undrafted,USA,Other D1,7
5,Drew Timme,24,-6.7,24,-6.700000,Gonzaga,0,Undrafted,Undrafted,USA,Top D1,2
7,Cameron Reynolds,23,-2.5,23,-2.500000,Tulane,0,Undrafted,Undrafted,USA,Other D1,7
9,Henry Sims,23,-2.2,23,-3.100000,Georgetown,0,Undrafted,Undrafted,USA,Top D1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
708,Davion Mitchell,23,-2.6,26,-2.933333,Baylor,9,1st,1-14,USA,Top D1,2
713,Josh Jackson,20,0.1,22,-2.366667,Kansas,4,1st,1-14,USA,Top D1,2
721,Lonnie Walker IV,21,-1.4,24,-2.100000,Miami,18,1st,15-30,USA,Top D1,2
732,Emmanuel Mudiay,19,-1.9,23,-2.366667,Guangdong,7,1st,1-14,China,CBA,9


## Adding Player Tier Column

In [284]:
player_tiers = []

for max_bpm in peaked_df['Top 3 BPM Avg']:
    if max_bpm >= 10:
        player_tiers.append('All Time Great')
    elif max_bpm < 10 and max_bpm >= 7.5:
        player_tiers.append('MVP')
    elif max_bpm < 7.5 and max_bpm >= 5:
        player_tiers.append('All-NBA')
    elif max_bpm < 5 and max_bpm >= 3.7:
        player_tiers.append('All-Star')
    elif max_bpm < 3.7 and max_bpm >= 2:
        player_tiers.append('Good Starter')
    elif max_bpm < 2 and max_bpm >= 0:
        player_tiers.append('Role Player')  
    elif max_bpm < 0 and max_bpm >= -2:
        player_tiers.append('Replacement Level')   
    else:
        player_tiers.append('Benchwarmer')     

peaked_df['Player Tier'] = player_tiers   

In [287]:
peaked_df['Player Tier'].value_counts()

Player Tier
Benchwarmer          324
Replacement Level    270
Role Player          239
Good Starter         130
All-Star              25
All-NBA               21
MVP                   12
All Time Great         7
Name: count, dtype: int64

In [289]:
peaked_df[peaked_df['Player Tier']=='Good Starter']

,Player,Age Enter,Max BPM,Age Peaked,Top 3 BPM Avg,College,Draft Pick,Round,Round Bin,Country,League,League Diff,Player Tier
12,Dru Smith,27,2.1,27,2.100000,Missouri,0,Undrafted,Undrafted,USA,Top D1,2,Good Starter
15,Josh Minott,22,2.2,22,2.200000,Memphis,45,2nd,45-60,USA,Top D1,2,Good Starter
68,Mike Miller,23,3.0,25,2.466667,Florida,5,1st,1-14,USA,Top D1,2,Good Starter
85,Chris Andersen,23,3.0,32,2.500000,Blinn J.C.,0,Undrafted,Undrafted,USA,JUCO,29,Good Starter
112,Collin Gillespie,25,2.5,25,2.500000,Villanova,0,Undrafted,Undrafted,USA,Top D1,2,Good Starter
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010,Jrue Holiday,19,3.4,29,3.233333,UCLA,17,1st,15-30,USA,Top D1,2,Good Starter
1012,Julius Randle,21,3.8,26,3.133333,Kentucky,7,1st,1-14,USA,Top D1,2,Good Starter
1022,CJ McCollum,22,3.9,29,3.000000,Lehigh,10,1st,1-14,USA,Other D1,7,Good Starter
1025,Tobias Harris,19,3.7,27,2.400000,Tennessee,19,1st,15-30,USA,Top D1,2,Good Starter


Final column on how thier careers have turned out considering their draft expecation

In [291]:
met_exp = []

for i in range(len(peaked_df)):
    tier = peaked_df.loc[i,'Player Tier']
    draft_pos = peaked_df.loc[i,'Round Bin']
    
    if draft_pos == '1-14' and tier in ['All-NBA','All Time Great']:
        met_exp.append('Exceeded')
    elif draft_pos == '1-14' and tier in ['Good Starter','All-Star']:
        met_exp.append('Met')
    elif draft_pos == '1-14' and tier == 'Role Player':
        met_exp.append('Below')   
    elif draft_pos == '1-14' and tier == 'Benchwarmer':
        met_exp.append('Bust')
        
    elif draft_pos == '14-30' and tier in ['All-Star','All-NBA','All Time Great']:
        met_exp.append('Exceeded')
    elif draft_pos == '14-30' and tier in ['Role Player','Good Starter']:
        met_exp.append('Met')
    elif draft_pos == '14-30' and tier == 'Replacement Level':
        met_exp.append('Below')   
    elif draft_pos == '14-30' and tier == 'Benchwarmer':
        met_exp.append('Bust')
        
    elif draft_pos == '31-44' and tier in ['Good Starter','All-Star','All-NBA','All Time Great']:
        met_exp.append('Exceeded')
    elif draft_pos == '31-44' and tier in ['Role Player']:
        met_exp.append('Met')
    elif draft_pos == '31-44' and tier == 'Replacement Level':
        met_exp.append('Below')   
    elif draft_pos == '31-44' and tier == 'Benchwarmer':
        met_exp.append('Bust')
    
    elif draft_pos == '34-60' and tier in ['Good Starter','All-Star','All-NBA','All Time Great']:
        met_exp.append('Exceeded')
    elif draft_pos == '34-60' and tier in ['Replacement Level','Role Player']:
        met_exp.append('Met')   
    elif draft_pos == '34-60' and tier == 'Benchwarmer':
        met_exp.append('Below')
    
    elif draft_pos == 'Undrafted' and tier in ['Replacement Level','Role Player','Good Starter','All-Star','All-NBA','All Time Great']:
        met_exp.append('Exceeded')
    else: 
        met_exp.append('Met')   

peaked_df['Met Draft Expt'] = met_exp
    

In [296]:
peaked_df.to_csv('Data/peaks.csv')